In [1]:
#!/usr/bin/env bash

<span style="color: red; font-weight: bold">Use the following command to launch a server for this notebook:</span>

```bash
git clone git@github.com:input-output-hk/marlowe-cardano.git
cd marlowe-cardano/marlowe-runtime/
nix run ../marlowe-cli
```

Then navigate to the `examples/` folder in Jupyter and open this notebook.

# Demonstrating the Marlowe Transaction Notify Component of Marlowe Runtime

## Preliminaries

Record version numbers

In [2]:
marlowe-cli --version

marlowe-cli 0.0.10.0


In [3]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [4]:
git rev-parse HEAD

ac7d3d0865a3d53ad3577358103b9350a83ebbbc


### Setup the faucet.

Set the location of keys.

In [5]:
TREASURY=treasury

Set the faucet.

In [6]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDR"

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Select network

In a separate terminal, set up a tunnel to the Marlowe Runtime development server:
```bash
rm /tmp/preview.socket
ssh -NT \
  -L/tmp/preview.socket:/data/networks/preview/node.socket \
  -L 3717:127.0.0.1:23717 \
  -L 3718:127.0.0.1:23718 \
  -L 3719:127.0.0.1:23719 \
  -L 3721:127.0.0.1:23721 \
  -L 3723:127.0.0.1:23723 \
  54.202.238.5                                              
```

In [7]:
export CARDANO_NODE_SOCKET_PATH=/tmp/preview.socket
export CARDANO_TESTNET_MAGIC=2
MAGIC=(--testnet-magic 2)
echo "${MAGIC[@]}"

--testnet-magic 2


### Check that the reference script has been published

Check that the Marlowe semantics validator was published.

In [8]:
marlowe-cli transaction find-published


Searching for reference script at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"
{
    "marlowe": {
        "hash": "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#1"
    },
    "payout": {
        "hash": "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#2"
    }
}


### Participants

#### The Party

In [9]:
PARTY_SKEY="$TREASURY/john-fletcher.skey"
PARTY_VKEY="$TREASURY/john-fletcher.vkey"

Create the first party's keys, if necessary.

In [10]:
if [[ ! -e "$PARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_SKEY" --verification-key-file "$PARTY_VKEY"
fi
PARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$PARTY_VKEY")
echo "$PARTY_ADDR"

addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r


Fund the address.

In [11]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600 \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$PARTY_ADDR"

TxId "cc7e9006b2cfa406b8ccff339c8c95822249c141bfdeaada87ebf37f4b6a053d"


### Time computations

In [12]:
SECOND=1000
MINUTE=$((60 * SECOND))

In [13]:
NOW="$(($(date -u +%s) * SECOND))"
echo "$NOW"

1666626300000


## The Contract

Set the contract parameters.

In [14]:
MINIMUM_ADA=3000000

Create the contract.

In [15]:
yaml2json << EOI > notify-1.contract
timeout: $((NOW+15*MINUTE))
timeout_continuation: close
when:
- case:
    notify_if: true
  then: close
EOI
cat notify-1.contract

{"timeout":1666627200000,"timeout_continuation":"close","when":[{"case":{"notify_if":true},"then":"close"}]}


## Run the Contract

### Transaction 1. Create the contract

Build the transaction.

In [16]:
CONTRACT_ID=$(
marlowe create \
  --core-file notify-1.contract \
  --min-utxo "$MINIMUM_ADA" \
  --change-address "$PARTY_ADDR" \
  --manual-sign notify-1.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "CONTRACT_ID = $CONTRACT_ID"

CONTRACT_ID = 8f3cac0eff99cae8184e7ed4e9cee3b379077bf0e7bd4a9600c8dd4b07c87d15#1


Sign the transaction.

In [17]:
cardano-cli transaction sign \
  --tx-body-file notify-1.txbody \
  --out-file     notify-1.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [18]:
marlowe submit notify-1.tx

"{\"blockHeaderHash\":\"b42a33a2202b8ff74be8c7603bcdefb60d46771a6fc24124fcdefa94a97d7476\",\"blockNo\":306358,\"slotNo\":6623126}"


Watch the contract.

In [19]:
marlowe add "$CONTRACT_ID"

8f3cac0eff99cae8184e7ed4e9cee3b379077bf0e7bd4a9600c8dd4b07c87d15#1


In [20]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 8f3cac0eff99cae8184e7ed4e9cee3b379077bf0e7bd4a9600c8dd4b07c87d15 (creation)
ContractId:      8f3cac0eff99cae8184e7ed4e9cee3b379077bf0e7bd4a9600c8dd4b07c87d15#1
SlotNo:          6623126
BlockNo:         306358
BlockId:         b42a33a2202b8ff74be8c7603bcdefb60d46771a6fc24124fcdefa94a97d7476
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Notify TrueObs) Close)] 1666627200000 Close



View the contract's UTxO.

In [21]:
CONTRACT_ADDR=$(marlowe-cli contract address)
echo "$CONTRACT_ADDR"

addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv


In [22]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${CONTRACT_ID//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
8f3cac0eff99cae8184e7ed4e9cee3b379077bf0e7bd4a9600c8dd4b07c87d15     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "5356030346f46bd3dc9ecded3255cf4599bcd142b6dba6d046b900c7e1ff532b"


### Transaction 2. Make the notification

Making the choice closes the contract.

In [23]:
marlowe notify \
  --contract "$CONTRACT_ID" \
  --validity-lower-bound "$((NOW - 1 * MINUTE))" \
  --validity-upper-bound "$((NOW + 9 * MINUTE))" \
  --change-address "$PARTY_ADDR" \
  --manual-sign notify-2.txbody

"{\"txId\":\"02c79ba6ab0bee70e8b592153bb74399193c166c85f10fc24b52b2257bb24d89\"}"


Sign the transaction.

In [24]:
cardano-cli transaction sign \
  --tx-body-file notify-2.txbody \
  --out-file     notify-2.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [25]:
marlowe submit notify-2.tx

"{\"blockHeaderHash\":\"227775fcebfc6a088e2e148b240365cd3dbc2893d24c5561880e89c65d922b4d\",\"blockNo\":306359,\"slotNo\":6623134}"


There should be no UTxO at the contract address.

In [26]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${CONTRACT_ID//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


In [27]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 8f3cac0eff99cae8184e7ed4e9cee3b379077bf0e7bd4a9600c8dd4b07c87d15 (creation)
ContractId:      8f3cac0eff99cae8184e7ed4e9cee3b379077bf0e7bd4a9600c8dd4b07c87d15#1
SlotNo:          6623126
BlockNo:         306358
BlockId:         b42a33a2202b8ff74be8c7603bcdefb60d46771a6fc24124fcdefa94a97d7476
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Notify TrueObs) Close)] 1666627200000 Close

transaction 02c79ba6ab0bee70e8b592153bb74399193c166c85f10fc24b52b2257bb24d89 (close)
ContractId: 8f3cac0eff99cae8184e7ed4e9cee3b379077bf0e7bd4a9600c8dd4b07c87d15#1
SlotNo:     6623134
BlockNo:    306359
BlockId:    227775fcebfc6a088e2e148b240365cd3dbc2893d24c5561880e89c65d922b4d
Inputs:     [NormalInput INotify]




## Cleanup

Remove the contract from history tracking.

In [28]:
marlowe rm "$CONTRACT_ID"

8f3cac0eff99cae8184e7ed4e9cee3b379077bf0e7bd4a9600c8dd4b07c87d15#1


In [29]:
marlowe ls

Consolidate the UTxOs at the addresses.

In [30]:
marlowe-cli util clean \
  --change-address "$PARTY_ADDR" \
  --required-signer "$PARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "6165ad0ce888a8e45588a380f2e8ce1659ba9a9a1258aa4200127878cf8a6bf7"


Send the funds back to the faucet.

In [31]:
marlowe-cli transaction simple \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$PARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --required-signer "$PARTY_SKEY" \
  --change-address "$FAUCET_ADDR" \
  --out-file /dev/null \
  --submit 600

TxId "6f1a532a2ed76d4353ebbf300306dfb3ec46f8d6afe63c5a744f7a6d2492e3d2"


See that the funds have been returned to the faucet.

In [32]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
